In [123]:
import sqlite3
import pandas as pd
import numpy as np
import datetime
from jbfg.pandas_utils import *

In [124]:
schema = 'l2_jbwc_extnl'

In [345]:
##
sql = ''' 
select  a.cstno 
        , b.division_name
        , b.category_name
        , b.name
        , a.order_interval
        , a.order_hour
        , count(1) as cnt
from   l2_jbwc_extnl.train_x_past a
left join  l2_jbwc_extnl.products_info b
on     a.product_id = b.product_id
where 1=1
group by  a.cstno 
        , b.division_name
        , b.category_name
        , b.name
        , a.order_interval
        , a.order_hour
        
'''
Train_X = get_df(sql,schema)

In [126]:
Train_X

,cstno,division_name,category_name,name,order_interval,cnt
0,6821,produce,fresh herbs,Bunched Cilantro,4.0,1
1,79238,produce,fresh fruits,Organic Lemon,10.0,1
2,156416,produce,packaged vegetables fruits,Organic Raspberries,8.0,1
3,87638,produce,packaged vegetables fruits,Organic Baby Arugula,6.0,1
4,198029,produce,fresh fruits,Organic Blackberries,5.0,2
...,...,...,...,...,...,...
674344,155880,produce,fresh fruits,Granny Smith Apples,13.0,1
674345,60800,produce,fresh vegetables,Organic Broccoli,8.0,1
674346,95164,produce,fresh fruits,Organic Bartlett Pear,0.0,1
674347,72601,produce,fresh vegetables,Fresh Cauliflower,7.0,1


In [78]:
# train set 복사
train_a = Train_X

In [127]:
# a1 : 아이템기반 
item_pivot = train_a.pivot_table('cnt', index = 'name', columns = 'cstno',aggfunc = 'sum')
# a2 : user기반 
user_pivot = train_a.pivot_table('cnt', index = 'cstno', columns = 'name',aggfunc = 'sum')


In [129]:
user_pivot

name,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Bartlett Pears,Blueberries,Boneless Skinless Chicken Breasts,...,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Whole Milk,Yellow Onions
cstno,,,,,,,,,,,,,,,,,,,,,
10000,NaN,41.0,NaN,NaN,NaN,6.0,30.0,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,2.0,NaN,NaN,NaN,1.0,1.0
100001,20.0,NaN,NaN,4.0,NaN,41.0,NaN,NaN,3.0,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
100046,NaN,NaN,NaN,NaN,1.0,2.0,21.0,3.0,NaN,NaN,...,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
100071,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
100108,NaN,NaN,NaN,14.0,NaN,20.0,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,NaN,NaN,15.0,NaN,NaN,NaN,45.0,NaN,NaN,NaN,...,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99829,NaN,1.0,NaN,NaN,NaN,1.0,34.0,NaN,8.0,1.0,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,16.0,NaN,12.0
99887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
#결측치 → 0
item_pivot.fillna(0,inplace = True)
user_pivot.fillna(0,inplace = True)

In [132]:
from sklearn.metrics.pairwise import cosine_similarity

In [134]:
##아이템별 유사도값 추출
similarity_rate = cosine_similarity(item_pivot,item_pivot)
print(similarity_rate)

[[1.         0.04207933 0.0158606  ... 0.02399683 0.010068   0.03501198]
 [0.04207933 1.         0.03496759 ... 0.0388578  0.04494993 0.10951319]
 [0.0158606  0.03496759 1.         ... 0.07845386 0.09941159 0.04495969]
 ...
 [0.02399683 0.0388578  0.07845386 ... 1.         0.03337813 0.07322478]
 [0.010068   0.04494993 0.09941159 ... 0.03337813 1.         0.07547474]
 [0.03501198 0.10951319 0.04495969 ... 0.07322478 0.07547474 1.        ]]


In [135]:
##아이템별 유사도값 데이터베이스 생성 : 1에 가까울 수록 유사
similarity_df = pd.DataFrame(data = similarity_rate,index = item_pivot.index,columns = item_pivot.index)
similarity_df.iloc[:4,:4]

name,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic
name,,,,
100% Raw Coconut Water,1.000000,0.042079,0.015861,0.114267
100% Whole Wheat Bread,0.042079,1.000000,0.034968,0.147430
2% Reduced Fat Milk,0.015861,0.034968,1.000000,0.013863
Apple Honeycrisp Organic,0.114267,0.147430,0.013863,1.000000


In [138]:
train_a["order_interval_int"] =  train_a["order_interval"].apply(pd.to_numeric) 

In [139]:
train_a

,cstno,division_name,category_name,name,order_interval,cnt,order_interval_int,wght_train,wght_cnt
0,6821,produce,fresh herbs,Bunched Cilantro,4.0,1,4.0,26.0,26.0
1,79238,produce,fresh fruits,Organic Lemon,10.0,1,10.0,20.0,20.0
2,156416,produce,packaged vegetables fruits,Organic Raspberries,8.0,1,8.0,22.0,22.0
3,87638,produce,packaged vegetables fruits,Organic Baby Arugula,6.0,1,6.0,24.0,24.0
4,198029,produce,fresh fruits,Organic Blackberries,5.0,2,5.0,25.0,50.0
...,...,...,...,...,...,...,...,...,...
674344,155880,produce,fresh fruits,Granny Smith Apples,13.0,1,13.0,17.0,17.0
674345,60800,produce,fresh vegetables,Organic Broccoli,8.0,1,8.0,22.0,22.0
674346,95164,produce,fresh fruits,Organic Bartlett Pear,0.0,1,0.0,30.0,30.0
674347,72601,produce,fresh vegetables,Fresh Cauliflower,7.0,1,7.0,23.0,23.0


In [140]:
#재구매까지의 소요시간 interval max : 30
print('min interval :',min(train_a["order_interval_int"] ),'/ max interval : ', max(train_a["order_interval_int"] ))

min interval : 0.0 / max interval :  30.0


In [141]:
##invertval 기간이 짧을수록 가중치를 높게 줌 (max:30 - order_interval)
train_a["wght_train"] = 30 - train_a["order_interval_int"]
train_a["wght_cnt"] = train_a["cnt"] * train_a["wght_train"]

In [142]:
train_a.iloc[:4,:4]

,cstno,division_name,category_name,name
0,6821,produce,fresh herbs,Bunched Cilantro
1,79238,produce,fresh fruits,Organic Lemon
2,156416,produce,packaged vegetables fruits,Organic Raspberries
3,87638,produce,packaged vegetables fruits,Organic Baby Arugula


In [143]:
# 재구매율 높은 물건에 가중치를 줬을 때의 유사도 
item_pivot2 = train_a.pivot_table('wght_cnt', index = 'name', columns = 'cstno')
item_pivot2.fillna(0,inplace = True)
similarity_rate2 = cosine_similarity(item_pivot2,item_pivot2)
similarity_df2 = pd.DataFrame(data = similarity_rate2,index = item_pivot2.index,columns = item_pivot2.index)

In [144]:
similarity_df2.iloc[:4,:4]

name,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic
name,,,,
100% Raw Coconut Water,1.000000,0.088943,0.021362,0.187333
100% Whole Wheat Bread,0.088943,1.000000,0.046836,0.219938
2% Reduced Fat Milk,0.021362,0.046836,1.000000,0.046360
Apple Honeycrisp Organic,0.187333,0.219938,0.046360,1.000000


In [145]:
aa = similarity_df2["Whole Milk"].sort_values(ascending = False)
aa

name
Whole Milk                                                        1.000000
Organic Fuji Apple                                                0.169221
Half & Half                                                       0.158835
Red Peppers                                                       0.157423
Small Hass Avocado                                                0.151885
                                                                    ...   
Organic Broccoli Florets                                          0.028292
Organic Grade A Free Range Large Brown Eggs                       0.027678
100% Raw Coconut Water                                            0.024084
Organic Sticks Low Moisture Part Skim Mozzarella String Cheese    0.023631
Organic Reduced Fat Milk                                          0.020331
Name: Whole Milk, Length: 100, dtype: float64

In [154]:
## 비교
print("기본 유사도 matrix 부여시 추천 item",recommand_item("Whole Milk"))
print("----------------------------")
print("기본 유사도 matrix + interval 가중치 부여시 추천 item: ",recommand_item2("Whole Milk"))


기본 유사도 matrix 부여시 추천 item Banana
----------------------------
기본 유사도 matrix + interval 가중치 부여시 추천 item:  Organic Fuji Apple


In [156]:
## user에 item 유사도 매트릭스 부여시, 구매할 확률 matrix 생성
def predict_rating(rating_arr, item_sim_arr) : 
    # rating_arr : u*i / itme_sim_arr : i*i
    sum_sr = rating_arr @ item_sim_arr
    sum_s_abs = np.array([np.abs(item_sim_arr).sum(axis=1)])
    
    rating_pred = sum_sr / sum_s_abs
    
    return rating_pred                          

In [157]:
rating_pred = predict_rating(user_pivot.values,similarity_df.values)
rating_pred2 = predict_rating(user_pivot.values,similarity_df2.values)

In [158]:
rating_pred_matrix = pd.DataFrame(data = rating_pred, index = user_pivot.index, columns = user_pivot.columns)
rating_pred_matrix2 = pd.DataFrame(data = rating_pred2, index = user_pivot.index, columns = user_pivot.columns)

rating_pred_matrix.head()

name,100% Raw Coconut Water,100% Whole Wheat Bread,2% Reduced Fat Milk,Apple Honeycrisp Organic,Asparagus,Bag of Organic Bananas,Banana,Bartlett Pears,Blueberries,Boneless Skinless Chicken Breasts,...,Sparkling Natural Mineral Water,Sparkling Water Grapefruit,Spring Water,Strawberries,Uncured Genoa Salami,Unsalted Butter,Unsweetened Almondmilk,Unsweetened Original Almond Breeze Almond Milk,Whole Milk,Yellow Onions
cstno,,,,,,,,,,,,,,,,,,,,,
10000,3.655858,7.783275,2.790816,4.309610,3.794886,4.157893,4.601562,3.593380,3.529192,3.599067,...,3.430279,3.628894,3.609238,3.953424,4.294529,3.747052,3.777726,3.559646,3.300478,3.745350
100001,7.053154,4.364750,1.962156,4.972397,4.039647,6.070707,3.264275,2.769543,3.181989,3.190818,...,4.847198,3.580057,4.627970,3.438662,4.144705,3.439781,4.323425,3.172637,2.698949,3.560848
100046,2.185232,2.376341,2.443578,2.365400,2.702256,2.303705,3.548911,3.400767,2.767760,2.819025,...,2.292095,2.642956,2.341831,2.994119,2.373714,2.694418,2.619953,2.715806,2.478473,2.895863
100071,2.505717,2.303169,0.953309,2.579906,2.539685,2.877937,2.094468,1.627176,1.718237,2.216546,...,2.275867,1.925360,3.064849,1.953246,2.361092,2.242755,2.784300,1.738747,1.699230,2.474645
100108,2.245140,2.160776,0.904728,2.812081,1.949608,2.845533,1.710584,1.428093,1.403411,1.661292,...,2.002785,1.787883,2.206026,1.639862,1.960914,1.704002,2.102399,1.854734,1.255012,1.728217


In [159]:
rating_pred_matrix.loc["10000"].sort_values(ascending = False)[:1].index.values[0]

'100% Whole Wheat Bread'

In [160]:
def recommand_item_1(cstno) : 
    return rating_pred_matrix.loc[cstno].sort_values(ascending = False)[:1].index.values[0]
def recommand_item_2(cstno) : 
    return rating_pred_matrix2.loc[cstno].sort_values(ascending = False)[:1].index.values[0]


In [161]:
print(recommand_item_1("10000"))
print(recommand_item_2("10000"))


100% Whole Wheat Bread
100% Whole Wheat Bread


In [163]:
#####EAD1 : 고객별 장바구니 담는 순서와 재구매 상품의 유사도

In [164]:
a1 = ''' 
SELECT  A.*
        , row_number() OVER (PARTITION BY cstno ORDER BY CNT DESC)   AS rnk
FROM(
    select  a.cstno
            , ADD_RN
            , b.name
            , count(1) AS CNT        
    from   (
        select  a.*,
                dense_rank() OVER (PARTITION BY cstno ORDER BY cast(order_number as integer)  ASC)     AS order_rn,
                row_number() OVER (PARTITION BY cstno, order_id ORDER BY cast(add_order as integer) ASC)     AS add_rn
                
        from    l2_jbwc_extnl.train_x_past a
        where 1 = 1 
        order by order_rn, add_rn
    ) a
    left join  l2_jbwc_extnl.products_info b
    on     a.product_id = b.product_id
    WHERE 1=1
    -- AND CSTNO = '10000'
    group by a.cstno
            , ADD_RN
            , b.name
    
    ORDER BY CNT DESC

)A
'''
A1 = get_df(a1,schema)

In [165]:
A1

,cstno,add_rn,name,cnt,rnk
0,10000,1,100% Whole Wheat Bread,23,1
1,10000,2,Organic Strawberries,16,2
2,10000,1,Organic Strawberries,15,3
3,10000,2,100% Whole Wheat Bread,14,4
4,10000,4,Fresh Cauliflower,8,5
...,...,...,...,...,...
529334,99901,3,Blueberries,1,19
529335,99901,4,Organic Small Bunch Celery,1,20
529336,99901,3,Organic Raspberries,1,21
529337,99901,5,Organic Strawberries,1,22


In [105]:
# 고객별로 장바구니에 첫번쨰 담은 빈도수가 가장 높은 물건구하기
A2 = A1[A1['rnk']==1]

In [308]:
## 장바구니 담은 순서별 같은 상품을 담을 확률 
p1 = A1.pivot_table('cnt', index = 'add_rn', aggfunc = 'sum')
p2 = A2.pivot_table('cnt', index = 'add_rn', aggfunc = 'sum')
p2['rate'] = 0
for i in p2.index :    
    p2['rate'].loc[i] =  round(p2.loc[i].values[0] / p1.loc[i].values[0] *100 ,2)

/opt/conda/envs/python3.6/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [310]:
## 같은고객 이며, 장바구니 담은 순서가 1번일때 31%의 확률로 재구매할 확률이 높음
p2

,cnt,rate
add_rn,,
1,108792,31.84
2,6760,2.48
3,1165,0.57
4,244,0.17
5,54,0.05
6,28,0.04
8,7,0.02


In [311]:
A2['rate'] = A2.cnt/A1[A1.add_rn == 1].pivot_table('cnt', index = 'cstno', aggfunc = 'sum').cnt.values*100

In [312]:
A2.iloc[:4,:6]

,cstno,add_rn,name,cnt,rnk,rate
0,10000,1,100% Whole Wheat Bread,23,1,34.848485
151,100001,1,Organic Strawberries,9,1,14.285714
305,100046,1,Michigan Organic Kale,20,1,32.258065
409,100071,1,Organic Romaine Lettuce,9,1,17.647059


In [315]:
# 같은 고객이 장바구니 첫번째에 담은 물건이 같을 확률이 50% 이상일 때, 그 고객이 첫번째에 담은 상품은 첫번째 가장 많이 담은 상품을 추천하도록
len(A2[A2['rate']>=50]) / len(A2)

0.1740911418330773

In [316]:
def recommand_item_3(cstno, add_rn) : 
    
    if add_rn == 1 and  round(A2[A2['cstno'] == cstno]['rate'].values[0],0) >= 50:
        a =  A2[A2['cstno'] == cstno]['name'].values[0]
    # if add_rn == 1 :
        # a =  A2[A2['cstno'] == cstno]['name'].values[0]
    else :
        a = rating_pred_matrix.loc[cstno].sort_values(ascending = False)[:1].index.values[0]
    return a

In [327]:
def recommand_item_4(cstno, add_rn) : 
    
    if add_rn == 1 and  round(A2[A2['cstno'] == cstno]['rate'].values[0],0) >= 50:
        a =  A2[A2['cstno'] == cstno]['name'].values[0]
    # if add_rn == 1 :
        # a =  A2[A2['cstno'] == cstno]['name'].values[0]
    else :
        a = rating_pred_matrix2.loc[cstno].sort_values(ascending = False)[:1].index.values[0]
    return a

In [385]:
## EDA 참고 : Soda를 한번이라도 구매한 고객이 10시에 Soda를 구매할 확률 54.9%
#  성능문제로 soda_cst 미리정의(Soda를 한번이라도 구매한 고객)
soda_cst = list(set(Train_X[(Train_X.name == 'Soda')].cstno))

def recommand_item_5(cstno, add_rn, order_hour) : 
    
    if (cstno in soda_cst) and (order_hour == 10):
        print('soda')
        a = 'Soda'
    elif add_rn == 1 and  round(A2[A2['cstno'] == cstno]['rate'].values[0],0) >= 50:
        a =  A2[A2['cstno'] == cstno]['name'].values[0]
    else :
        a = rating_pred_matrix.loc[cstno].sort_values(ascending = False)[:1].index.values[0]
    return a

In [371]:
recommand_item_5('100142',3,10)
# type(train_X_current_2['add_rn'])

'Soda'

In [318]:
####################################################################
# 예측 데이터와 실제데이터 비교

In [319]:
sql22 = ''' 
select  *
from   l2_jbwc_extnl.train_x_current a
'''
Train_X_current = get_df(sql22,schema)

sql23 = ''' 
select  a.product_id, b.name
from   l2_jbwc_extnl.train_y a
left join  l2_jbwc_extnl.products_info b
on     a.product_id = b.product_id
where 1=1
'''
Train_y = get_df(sql23,schema)
sql24 = ''' 
    select  a.*,
            dense_rank() OVER (PARTITION BY cstno ORDER BY cast(order_number as integer)  ASC)     AS order_rn,
            row_number() OVER (PARTITION BY cstno, order_id ORDER BY cast(add_order as integer) ASC)     AS add_rn
    from    l2_jbwc_extnl.train_x_current a
    where 1 = 1 

'''
train_X_current_2 = get_df(sql24,schema)

In [338]:
Train_X_current
# Train_y
pred_y = pd.DataFrame(columns = ['cstno','y','y2','y3','y4','y5','y6','y7'])
pred_y["cstno"] = Train_X_current["cstno"]
# pred_y["y"][pred_y["cstno"] == '41116']

In [339]:
train_X_current_2

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,order_rn,add_rn
0,1925217,10000,73,4,21,6.0,3,1,1,1
1,805264,100001,67,1,21,7.0,1,1,1,1
2,805264,100001,67,1,21,7.0,3,1,1,2
3,805264,100001,67,1,21,7.0,5,1,1,3
4,805264,100001,67,1,21,7.0,8,1,1,4
...,...,...,...,...,...,...,...,...,...,...
24419,2512023,99894,49,1,21,1.0,7,1,1,2
24420,2512023,99894,49,1,21,1.0,8,1,1,3
24421,2512023,99894,49,1,21,1.0,12,1,1,4
24422,2921725,99901,49,4,13,8.0,3,1,1,1


In [340]:
for j in Train_X_current["cstno"] : 
    pred_y["y"][pred_y["cstno"] == j] = recommand_item_1(j)
    pred_y["y2"][pred_y["cstno"] == j] = recommand_item_2(j)

In [341]:
for i in range(len(train_X_current_2)):
    pred_y["y3"][pred_y["cstno"] == train_X_current_2.loc[i]['cstno']] = recommand_item_3(train_X_current_2.loc[i]['cstno'], train_X_current_2.loc[i]['add_rn'])

In [342]:
for i in range(len(train_X_current_2)):
    pred_y["y4"][pred_y["cstno"] == train_X_current_2.loc[i]['cstno']] = recommand_item_4(train_X_current_2.loc[i]['cstno'], train_X_current_2.loc[i]['add_rn'])

In [395]:
for i in range(len(train_X_current_2)):
    pred_y["y5"][pred_y["cstno"] == train_X_current_2.loc[i]['cstno']] = recommand_item_5(train_X_current_2.loc[i]['cstno'], train_X_current_2.loc[i]['add_rn'],train_X_current_2.loc[i]['order_hour'])
    if (i%10000)== 0 :
        print(i)

0
10000
20000


In [364]:
pred_y["real_product_id"] = Train_y["product_id"]
pred_y["real_y"] = Train_y["name"]

In [396]:
print( 'item기반 : ' , sum(pred_y["y"]==pred_y["real_y"])/len(pred_y))
print( 'item기반+재구매가중치 : ' , sum(pred_y["y2"]==pred_y["real_y"])/len(pred_y))
print( 'item기반+첫번째선택상품 : ' , sum(pred_y["y3"]==pred_y["real_y"])/len(pred_y))
print( 'item기반+재구매가중치+첫번째선택상품 : ' , sum(pred_y["y4"]==pred_y["real_y"])/len(pred_y))  ## 안쓰는게 나을듯? 
print( 'item기반+첫번째선택상품+10시 Soda : ' , sum(pred_y["y5"]==pred_y["real_y"])/len(pred_y))  ## 해당조건 없어서 변동없음

item기반 :  0.11394529970520799
item기반+재구매가중치 :  0.11062888961677039
item기반+첫번째선택상품 :  0.11595152309204061
item기반+재구매가중치+첫번째선택상품 :  0.11279888634130364
item기반+첫번째선택상품+10시 Soda :  0.11595152309204061


In [122]:
# pred_y["add_rn"] = train_X_current_2["add_rn"]

In [393]:
[train_X_current_2['cstno'] == '121846'

SyntaxError: unexpected EOF while parsing (<ipython-input-393-4c1cd0b1808c>, line 1)

In [387]:
train_X_current_2.loc[0]['cstno']in list(soda_cst)

False